# Logistic Regression Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('log1').getOrCreate()

In [3]:
data = spark.read.csv('customer_churn.csv',inferSchema=True,header=True)

In [5]:
data.head(5)

[Row(Names='Cameron Williams', Age=42.0, Total_Purchase=11066.8, Account_Manager=0, Years=7.22, Num_Sites=8.0, Onboard_date='2013-08-30 07:00:40', Location='10265 Elizabeth Mission Barkerburgh, AK 89518', Company='Harvey LLC', Churn=1),
 Row(Names='Kevin Mueller', Age=41.0, Total_Purchase=11916.22, Account_Manager=0, Years=6.5, Num_Sites=11.0, Onboard_date='2013-08-13 00:38:46', Location='6157 Frank Gardens Suite 019 Carloshaven, RI 17756', Company='Wilson PLC', Churn=1),
 Row(Names='Eric Lozano', Age=38.0, Total_Purchase=12884.75, Account_Manager=0, Years=6.67, Num_Sites=12.0, Onboard_date='2016-06-29 06:20:07', Location='1331 Keith Court Alyssahaven, DE 90114', Company='Miller, Johnson and Wallace', Churn=1),
 Row(Names='Phillip White', Age=42.0, Total_Purchase=8010.76, Account_Manager=0, Years=6.71, Num_Sites=10.0, Onboard_date='2014-04-22 12:43:12', Location='13120 Daniel Mount Angelabury, WY 30645-4695', Company='Smith Inc', Churn=1),
 Row(Names='Cynthia Norton', Age=37.0, Total_P

In [6]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [7]:
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

### Selecting the features

In [8]:
my_cols = data.select(['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Company',
 'Churn',])

In [9]:
log_data = my_cols.na.drop()

In [10]:
log_data.show()

+----+--------------+---------------+-----+---------+--------------------+-----+
| Age|Total_Purchase|Account_Manager|Years|Num_Sites|             Company|Churn|
+----+--------------+---------------+-----+---------+--------------------+-----+
|42.0|       11066.8|              0| 7.22|      8.0|          Harvey LLC|    1|
|41.0|      11916.22|              0|  6.5|     11.0|          Wilson PLC|    1|
|38.0|      12884.75|              0| 6.67|     12.0|Miller, Johnson a...|    1|
|42.0|       8010.76|              0| 6.71|     10.0|           Smith Inc|    1|
|37.0|       9191.58|              0| 5.56|      9.0|          Love-Jones|    1|
|48.0|      10356.02|              0| 5.12|      8.0|        Kelly-Warren|    1|
|44.0|      11331.58|              1| 5.23|     11.0|   Reynolds-Sheppard|    1|
|32.0|       9885.12|              1| 6.92|      9.0|          Singh-Cole|    1|
|43.0|       14062.6|              1| 5.46|     11.0|           Lopez PLC|    1|
|40.0|       8066.94|       

## Dealing with Categorical Variables [Company]

In [11]:
from pyspark.ml.feature import (VectorAssembler,VectorIndexer,
                                OneHotEncoder,StringIndexer)

In [12]:
company_indexer = StringIndexer(inputCol='Company',outputCol='CompanyIndex')
company_encoder = OneHotEncoder(inputCol='CompanyIndex',outputCol='CompanyVec')

In [14]:
assembler = VectorAssembler(inputCols=['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'CompanyVec',],outputCol='features')

In [15]:
from pyspark.ml.classification import LogisticRegression

In [16]:
from pyspark.ml import Pipeline

In [18]:
log_reg_churn = LogisticRegression(featuresCol='features',labelCol='Churn')

In [19]:
pipeline = Pipeline(stages=[company_indexer,company_encoder,
                                                      assembler,log_reg_churn])

In [20]:
train_data, new_customers = log_data.randomSplit([0.7,.3])

In [21]:
new_customers.show()

+----+--------------+---------------+-----+---------+--------------------+-----+
| Age|Total_Purchase|Account_Manager|Years|Num_Sites|             Company|Churn|
+----+--------------+---------------+-----+---------+--------------------+-----+
|26.0|       8787.39|              1| 5.42|     11.0|Tate, Hall and Mo...|    1|
|26.0|       8939.61|              0| 4.54|      7.0|Brown, Johnson an...|    0|
|29.0|       9617.59|              0| 5.49|      8.0|         Mendoza Inc|    0|
|30.0|       6744.87|              0| 5.14|      9.0|          Morris LLC|    0|
|30.0|      10183.98|              1| 5.14|      9.0|Smith, Salazar an...|    0|
|30.0|      12788.37|              0| 4.31|     10.0|      Everett-Taylor|    0|
|31.0|       5387.75|              0| 6.83|      9.0|       Morales-Miles|    0|
|31.0|       9574.89|              0| 7.34|      8.0|Anderson, Brown a...|    0|
|31.0|      10058.87|              1| 6.54|      7.0|          Knox-Davis|    0|
|31.0|       10182.6|       

### Traning and Testing the Model

In [22]:
fit_model = pipeline.fit(train_data)

In [24]:
predictions= fit_model.transform(new_customers)

In [25]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [26]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                       labelCol='Churn')

In [28]:
predictions.select('Churn','prediction').show()

Py4JJavaError: An error occurred while calling o307.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 51.0 failed 1 times, most recent failure: Lost task 0.0 in stage 51.0 (TID 51, DESKTOP-FN0L0QJ, executor driver): org.apache.spark.SparkException: Failed to execute user defined function(StringIndexerModel$$Lambda$2768/0x0000000101207840: (string) => double)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:726)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$getByteArrayRdd$1(SparkPlan.scala:321)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitionsInternal$2(RDD.scala:872)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitionsInternal$2$adapted(RDD.scala:872)
	at org.a

In [30]:
AUC = my_eval.evaluate(predictions)

Py4JJavaError: An error occurred while calling o302.evaluate.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 52.0 failed 1 times, most recent failure: Lost task 0.0 in stage 52.0 (TID 52, DESKTOP-FN0L0QJ, executor driver): org.apache.spark.SparkException: Failed to execute user defined function(StringIndexerModel$$Lambda$2768/0x0000000101207840: (string) => double)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:726)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458

In [31]:
AUC

NameError: name 'AUC' is not defined